## This script loads the current model and performs an evaluation of it

### Initialize
First, initialize the model with all parameters


In [1]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
from model import Model
from loss import TripletLoss, ImprovedTripletLoss
from training_set import TrainingSet
from average_meter import AverageMeter
from data_splitter import DataSplitter
from mission_indices import MissionIndices
from database_parser import DatabaseParser

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import pyshtools
from pyshtools import spectralanalysis
from pyshtools import shio
from pyshtools import expand

import sys
import time
import math
import operator
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from tqdm.auto import tqdm
import scipy.stats as st
from scipy import spatial

%reload_ext autoreload
%autoreload 2

In [2]:
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True
n_features = 3
bandwidth = 100
net = Model(n_features, bandwidth).cuda()
restore = False
optimizer = torch.optim.SGD(net.parameters(), lr=5e-3, momentum=0.9)
batch_size = 12
num_workers = 12
descriptor_size = 256
net_input_size = 2*bandwidth
cache = 50
criterion = ImprovedTripletLoss(margin=2, alpha=0.5, margin2=0.2)
writer = SummaryWriter()
stored_model = './net_params_arche_high_res_big.pkl'
net.load_state_dict(torch.load(stored_model))
#summary(net, input_size=[(2, 200, 200), (2, 200, 200), (2, 200, 200)])

<All keys matched successfully>

In [15]:
indices = np.arange(0,10,2)
print(indices)

[0 2 4 6 8]


Initialize the data source

In [34]:
dataset_path = "/media/scratch/berlukas/spherical/koze_high_res/"
#dataset_path = "/home/berlukas/data/arche_low_res2/"
db_parser = DatabaseParser(dataset_path)

n_test_data = 2500
n_test_cache = n_test_data

idx = np.arange(0,n_test_data, 10)

ds_test = DataSource(dataset_path, n_test_cache, -1, False)
ds_test.load(n_test_data, idx, filter_clusters=True)
n_test_data = len(ds_test.anchors)

Reading missions db from /media/scratch/berlukas/spherical/koze_high_res/missions.csv
Read 12378 entries.
Loading anchors from:	/media/scratch/berlukas/spherical/koze_high_res//training_anchor_pointclouds/ and /media/scratch/berlukas/spherical/koze_high_res//training_anchor_sph_images/



Loading positives from:	/media/scratch/berlukas/spherical/koze_high_res//training_positive_pointclouds/ and /media/scratch/berlukas/spherical/koze_high_res//training_positive_sph_images/



Done loading dataset.
	Anchor point clouds total: 	46
	Anchor images total: 		46
	Anchor poses total: 		46
	Positive point clouds total: 	46
	Positive images total: 		46
	Positive poses total: 		46


In [35]:
test_set = TrainingSet(restore, bandwidth)
test_set.generateAll(ds_test)

Generating features from 0 to 46
Generating anchor spheres



Processing time in total 10.295600652694702 for 46 anchors.
Generating positive spheres



Processing time in total 24.101486444473267 for 92 positives.
Generated all pcl features
Processing time in total 24.101486444473267 for 92 items.
Processing time avg is 0.26197


In [36]:
# hack for removing the images
#test_set.anchor_features = test_set.anchor_features[:,0:2,:,:]
#test_set.positive_features = test_set.positive_features[:,0:2,:,:]
#test_set.negative_features = test_set.negative_features[:,0:2,:,:]


n_test_set = len(test_set)
print("Total size: ", n_test_set)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False, num_workers=1, pin_memory=True, drop_last=False)

Total size:  46


# Generate the descriptors for the positive samples

In [37]:
net.eval()
n_iter = 0

anchor_embeddings = np.empty(1)
positive_embeddings = np.empty(1)
with torch.no_grad():
    for batch_idx, (data1, data2) in enumerate(test_loader):
        embedded_a, embedded_p, embedded_n = net(data1.cuda().float(), data2.cuda().float(), data2.cuda().float())
        dist_to_pos, dist_to_neg, loss, loss_total = criterion(embedded_a, embedded_p, embedded_n)
        
        anchor_embeddings = np.append(anchor_embeddings, embedded_a.cpu().data.numpy().reshape([1,-1]))
        positive_embeddings = np.append(positive_embeddings, embedded_p.cpu().data.numpy().reshape([1,-1]))
        n_iter = n_iter + 1
        
desc_anchors = anchor_embeddings[1:].reshape([n_test_set, descriptor_size])        
desc_positives = positive_embeddings[1:].reshape([n_test_set, descriptor_size])      

## New testing pipeline (location based)

In [38]:
print(f'Running test pipeline for a map size of {len(desc_anchors)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_anchors)
p_norm = 2
max_pos_dist = 5.0
anchor_poses = ds_test.anchor_poses
positive_poses = ds_test.positive_poses

for n_nearest_neighbors in tqdm(range(1,21)):    
    loc_count = 0
    match_count = 0
    for idx in range(n_test_set):
        cur_positive_pos = positive_poses[idx,5:8]
        diff = np.subtract(anchor_poses[:,5:8], cur_positive_pos)
        distances = np.linalg.norm(diff, axis=1)
        if (np.count_nonzero(distances <= max_pos_dist) <= 2):            
            continue
        match_count = match_count + 1
            
        
        nn_dists, nn_indices = tree.query(desc_positives[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(anchor_poses[nn_i,5:8], cur_positive_pos)
            if (dist <= max_pos_dist):
                loc_count = loc_count + 1;
                break
                
    loc_precision = (loc_count*1.0) / match_count    
    print(f'recall {loc_precision} for {n_nearest_neighbors} neighbors')
    #print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/Location', loc_precision, n_nearest_neighbors)    

Running test pipeline for a map size of 46 descriptors.


recall 0.34782608695652173 for 1 neighbors
recall 0.34782608695652173 for 2 neighbors
recall 0.3695652173913043 for 3 neighbors
recall 0.3695652173913043 for 4 neighbors
recall 0.3695652173913043 for 5 neighbors
recall 0.3695652173913043 for 6 neighbors
recall 0.3695652173913043 for 7 neighbors
recall 0.3695652173913043 for 8 neighbors
recall 0.3695652173913043 for 9 neighbors
recall 0.3695652173913043 for 10 neighbors
recall 0.3695652173913043 for 11 neighbors
recall 0.391304347826087 for 12 neighbors
recall 0.6956521739130435 for 13 neighbors
recall 0.717391304347826 for 14 neighbors
recall 0.9782608695652174 for 15 neighbors
recall 0.9782608695652174 for 16 neighbors
recall 1.0 for 17 neighbors
recall 1.0 for 18 neighbors
recall 1.0 for 19 neighbors
recall 1.0 for 20 neighbors



In [28]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0
max_anchor_dist = 1
anchor_poses = ds_test.anchor_poses
positive_poses = ds_test.positive_poses
assert len(anchor_poses) == len(positive_poses)

for n_nearest_neighbors in tqdm(range(1,21)):    
    loc_count = 0
    for idx in range(n_test_set):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                loc_count = loc_count + 1;
                break
                
    loc_precision = (loc_count*1.0) / n_test_set    
    #print(f'recall {loc_precision} for {n_nearest_neighbors} neighbors')
    print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/Location', loc_precision, n_nearest_neighbors)    

Running test pipeline for a map size of 58 descriptors.


0.3793103448275862
0.39655172413793105
0.6379310344827587
0.6551724137931034
0.6551724137931034
0.6551724137931034
0.6551724137931034
0.6724137931034483
0.6896551724137931
0.6896551724137931
0.6896551724137931
0.6896551724137931
0.6896551724137931
0.8620689655172413
0.9310344827586207
0.9310344827586207
0.9655172413793104
0.9655172413793104
0.9655172413793104
0.9655172413793104



## Place Voting using Global Spectral Analysis


In [47]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
start = time.time()
tree = spatial.KDTree(desc_anchors)
end = time.time()
print(f'Duration for building the kd-tree {(end - start)}s')   
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds_test.anchor_poses
anchor_clouds = ds_test.anchors
anchor_features = test_set.anchor_features

positive_poses = ds_test.positive_poses
positive_clouds = ds_test.positives
positive_features = test_set.anchor_features

n_bands = 15
tapers, eigenvalues, taper_order = spectralanalysis.SHReturnTapers(2.01, 15)
for n_nearest_neighbors in tqdm(range(10,21)):        
    #n_nearest_neighbors = 16
    n_matches = 0    
    loc_count = 0
    almost_loc_count = 0
    hard_loc_count = 0
    no_loc_count = 0
    
    fused_loc_count = 0
    fused_almost_loc_count = 0
    fused_hard_loc_count = 0
    fused_no_loc_count = 0
    
    final_count = 0
    dur_neighbor_processing_s = 0
    dur_s2_s = 0
    dur_spectrum_s = 0
    for idx in range(0, n_test_set):        
    #for idx in range(0, 200):        
        start = time.time()
        nn_dists, nn_indices = tree.query(desc_positives[idx,:], p = p_norm, k = n_nearest_neighbors)                
        end = time.time()
        dur_neighbor_processing_s = dur_neighbor_processing_s + (end - start)

        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        z_scores_fused = [0] * n_nearest_neighbors
        z_scores_range = [0] * n_nearest_neighbors        
        z_scores_intensity = [0] * n_nearest_neighbors
        z_scores_image = [0] * n_nearest_neighbors                     
        n_true_matches = 0   
        contains_match = False
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_test_set):
                print(f'ERROR: index {nn_i} is outside of {n_test_set}')
                break;

            dist = spatial.distance.euclidean(anchor_poses[nn_i,5:8], positive_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True   
                n_true_matches = n_true_matches + 1                

            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
            a_intensity = anchor_features[idx][1,:,:]
            p_intensity = positive_features[nn_i][1,:,:]
            a_img = anchor_features[idx][2,:,:]
            p_img = positive_features[nn_i][2,:,:]

            start_s2 = time.time()
            a_range_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_range_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)

            a_intensity_coeffs = pyshtools.expand.SHExpandDH(a_intensity, sampling=1)
            p_intensity_coeffs = pyshtools.expand.SHExpandDH(p_intensity, sampling=1)

            a_img_coeffs = pyshtools.expand.SHExpandDH(a_img, sampling=1)
            p_img_coeffs = pyshtools.expand.SHExpandDH(p_img, sampling=1)
            end_s2 = time.time()
            dur_s2_s = dur_s2_s + (end_s2 - start_s2)


            start_spectrum = time.time()
            saa_range = spectralanalysis.spectrum(a_range_coeffs)            
            saa_intensity = spectralanalysis.spectrum(a_intensity_coeffs)    
            saa_img = spectralanalysis.spectrum(a_img_coeffs)    
            saa = np.empty([n_features, saa_range.shape[0]])
            saa[0,:] = saa_range
            saa[1,:] = saa_intensity
            saa[2,:] = saa_img
            #saa = np.mean(saa, axis=0)
            saa = np.amax(saa, axis=0)

            spp_range = spectralanalysis.spectrum(p_range_coeffs)            
            spp_intensity = spectralanalysis.spectrum(p_intensity_coeffs)    
            spp_img = spectralanalysis.spectrum(p_img_coeffs)    
            spp = np.empty([n_features, spp_range.shape[0]])
            spp[0,:] = spp_range
            spp[1,:] = spp_intensity
            spp[2,:] = spp_img
            #spp = np.mean(spp, axis=0)
            spp = np.amax(spp, axis=0)

            sap_range = spectralanalysis.cross_spectrum(a_range_coeffs, p_range_coeffs)            
            sap_intensity = spectralanalysis.cross_spectrum(a_intensity_coeffs, p_intensity_coeffs)    
            sap_img = spectralanalysis.cross_spectrum(a_img_coeffs, p_img_coeffs)    
            sap = np.empty([n_features, sap_range.shape[0]])
            sap[0,:] = sap_range
            sap[1,:] = sap_intensity
            sap[2,:] = sap_img
            #sap = np.mean(sap, axis=0)
            sap = np.amax(sap, axis=0)

            #saa = spectralanalysis.spectrum(a_coeffs)
            #spp = spectralanalysis.spectrum(p_coeffs)
            #sap = spectralanalysis.cross_spectrum(a_coeffs, p_coeffs)

            #admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)                                    
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap, saa, spp, tapers)
            end_spectrum = time.time()
            dur_spectrum_s = dur_spectrum_s + (end_spectrum - start_spectrum)            
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_fused[i] = z_scores_fused[i] + score  
                        
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_range, saa_range, spp_range, tapers)                        
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_range[i] = z_scores_range[i] + score
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_intensity, saa_intensity, spp_intensity, tapers)                            
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_intensity[i] = z_scores_intensity[i] + score                           
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)                            
            for l in range(0, n_bands):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_image[i] = z_scores_image[i] + score       
            

        #print(z_scores_range)
        #print(z_scores_intensity)
        #print(f'z_score > 2 = {np.sum(np.array(z_scores_range) > 3.8)} range, {np.sum(np.array(z_scores_intensity) > 20)} intensity')
        #print(f'true matches: {n_true_matches}')

        # normalize values
        z_scores_fused = np.array(z_scores_fused) / (n_bands)
        z_scores_range = np.array(z_scores_range) / (n_bands)
        z_scores_intensity = np.array(z_scores_intensity) / (n_bands)
        z_scores_image = np.array(z_scores_image) / (n_bands)
        
        n_matches = n_matches + 1
        max_index_fused, max_z_score_fused = max(enumerate(z_scores_fused), key=operator.itemgetter(1))
        max_index_range, max_z_score_range = max(enumerate(z_scores_range), key=operator.itemgetter(1))
        max_index_intensity, max_z_score_intensity = max(enumerate(z_scores_intensity), key=operator.itemgetter(1))        
        max_index_image, max_z_score_image = max(enumerate(z_scores_image), key=operator.itemgetter(1))
        
        
        max_index = max_index_range if max_z_score_range > max_z_score_intensity else max_index_intensity        
        max_score = max_z_score_range if max_z_score_range > max_z_score_intensity else max_z_score_intensity

        max_index = max_index if max_score > max_z_score_image else max_index_image

        matching_index = nn_indices[max_index]
        dist = spatial.distance.euclidean(anchor_poses[matching_index,5:8], positive_poses[idx,5:8])
        if (dist <= 5):
            loc_count = loc_count + 1;            
        elif (dist <= 8):
            almost_loc_count = almost_loc_count + 1
        elif (dist <= 11):
            hard_loc_count = hard_loc_count + 1
        else:
            no_loc_count = no_loc_count + 1
            
        matching_index = nn_indices[max_index_fused]
        dist = spatial.distance.euclidean(anchor_poses[matching_index,5:8], positive_poses[idx,5:8])
        if (dist <= 5):
            fused_loc_count = fused_loc_count + 1;            
        elif (dist <= 8):
            fused_almost_loc_count = fused_almost_loc_count + 1
        elif (dist <= 11):
            fused_hard_loc_count = fused_hard_loc_count + 1
        else:
            fused_no_loc_count = fused_no_loc_count + 1
            
        
    
    loc_precision = (loc_count*1.0) / n_matches    
    almost_loc_precision = (almost_loc_count*1.0) / n_matches    
    hard_loc_precision = (hard_loc_count*1.0) / n_matches    
    no_loc_precision = (no_loc_count*1.0) / n_matches    
    
    fused_loc_precision = (fused_loc_count*1.0) / n_matches    
    fused_almost_loc_precision = (fused_almost_loc_count*1.0) / n_matches    
    fused_hard_loc_precision = (fused_hard_loc_count*1.0) / n_matches    
    fused_no_loc_precision = (fused_no_loc_count*1.0) / n_matches    
    
    print(f'Recall loc: {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_test_set} correct matches.')
    print(f'Remaining recall: almost: {almost_loc_precision}, hard: {hard_loc_precision}, no {no_loc_precision}')
    print(f'[FUSED] Recall loc: {fused_loc_precision}, almost: {fused_almost_loc_precision}, hard: {fused_hard_loc_precision}, no {fused_no_loc_precision}')    
    print('-----------------------------------------------------------------------------------------------------------------')
    #print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/WindowedVoting', loc_precision, n_nearest_neighbors)
    #print(f'Duration: {dur_neighbor_processing_s/n_test_set}s')    
    #print(f'Duration S^2 Transform: {dur_s2_s/n_test_set}s')
    #print(f'Duration Spectrum: {dur_spectrum_s/n_test_set}s')


Running test pipeline for a map size of 46 descriptors.
Duration for building the kd-tree 0.002602815628051758s


Recall loc: 0.34782608695652173 for 10 neighbors with 46/46 correct matches.
Remaining recall: almost: 0.13043478260869565, hard: 0.17391304347826086, no 0.34782608695652173
[FUSED] Recall loc: 0.34782608695652173, almost: 0.10869565217391304, hard: 0.17391304347826086, no 0.3695652173913043
-----------------------------------------------------------------------------------------------------------------
Recall loc: 0.34782608695652173 for 11 neighbors with 46/46 correct matches.
Remaining recall: almost: 0.13043478260869565, hard: 0.17391304347826086, no 0.34782608695652173
[FUSED] Recall loc: 0.34782608695652173, almost: 0.10869565217391304, hard: 0.17391304347826086, no 0.3695652173913043
-----------------------------------------------------------------------------------------------------------------
Recall loc: 0.3695652173913043 for 12 neighbors with 46/46 correct matches.
Remaining recall: almost: 0.13043478260869565, hard: 0.15217391304347827, no 0.34782608695652173
[FUSED] Recal

In [ ]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
start = time.time()
tree = spatial.KDTree(desc_positives)
end = time.time()
print(f'Duration for building the kd-tree {(end - start)}s')   
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds_test.anchor_poses
anchor_clouds = ds_test.anchors
anchor_features = test_set.anchor_features

positive_poses = ds_test.positive_poses
positive_clouds = ds_test.positives
positive_features = test_set.anchor_features

tapers, eigenvalues, taper_order = spectralanalysis.SHReturnTapers(2.01, 1)
#for n_nearest_neighbors in tqdm(range(19,20)):        
n_nearest_neighbors = 16
n_matches = 0    
loc_count = 0    
final_count = 0
dur_neighbor_processing_s = 0
dur_s2_s = 0
dur_spectrum_s = 0
for idx in range(0, n_test_set):        
#for idx in range(0, 200):        
    start = time.time()
    nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)                
    end = time.time()
    dur_neighbor_processing_s = dur_neighbor_processing_s + (end - start)

    nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
    z_scores_range = [0] * n_nearest_neighbors
    z_scores_intensity = [0] * n_nearest_neighbors
    z_scores_image = [0] * n_nearest_neighbors                     
    n_true_matches = 0   
    contains_match = False
    for i in range(0, n_nearest_neighbors):
        nn_i = nn_indices[i]            
        if (nn_i >= n_test_set):
            print(f'ERROR: index {nn_i} is outside of {n_data}')
            break;

        dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            contains_match = True   
            n_true_matches = n_true_matches + 1                

        a_range = anchor_features[idx][0,:,:]
        p_range = positive_features[nn_i][0,:,:]
        a_intensity = anchor_features[idx][1,:,:]
        p_intensity = positive_features[nn_i][1,:,:]
        a_img = anchor_features[idx][2,:,:]
        p_img = positive_features[nn_i][2,:,:]

        start_s2 = time.time()
        a_range_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
        p_range_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)

        a_intensity_coeffs = pyshtools.expand.SHExpandDH(a_intensity, sampling=1)
        p_intensity_coeffs = pyshtools.expand.SHExpandDH(p_intensity, sampling=1)

        a_img_coeffs = pyshtools.expand.SHExpandDH(a_img, sampling=1)
        p_img_coeffs = pyshtools.expand.SHExpandDH(p_img, sampling=1)
        end_s2 = time.time()
        dur_s2_s = dur_s2_s + (end_s2 - start_s2)


        start_spectrum = time.time()
        saa_range = spectralanalysis.spectrum(a_range_coeffs)            
        saa_intensity = spectralanalysis.spectrum(a_intensity_coeffs)    
        saa_img = spectralanalysis.spectrum(a_img_coeffs)    
        saa = np.empty([n_features, saa_range.shape[0]])
        saa[0,:] = saa_range
        saa[1,:] = saa_intensity
        saa[2,:] = saa_img
        #saa = np.mean(saa, axis=0)
        saa = np.amax(saa, axis=0)

        spp_range = spectralanalysis.spectrum(p_range_coeffs)            
        spp_intensity = spectralanalysis.spectrum(p_intensity_coeffs)    
        spp_img = spectralanalysis.spectrum(p_img_coeffs)    
        spp = np.empty([n_features, spp_range.shape[0]])
        spp[0,:] = spp_range
        spp[1,:] = spp_intensity
        spp[2,:] = spp_img
        #spp = np.mean(spp, axis=0)
        spp = np.amax(spp, axis=0)

        sap_range = spectralanalysis.cross_spectrum(a_range_coeffs, p_range_coeffs)            
        sap_intensity = spectralanalysis.cross_spectrum(a_intensity_coeffs, p_intensity_coeffs)    
        sap_img = spectralanalysis.cross_spectrum(a_img_coeffs, p_img_coeffs)    
        sap = np.empty([n_features, sap_range.shape[0]])
        sap[0,:] = sap_range
        sap[1,:] = sap_intensity
        sap[2,:] = sap_img
        #sap = np.mean(sap, axis=0)
        sap = np.amax(sap, axis=0)

        #saa = spectralanalysis.spectrum(a_coeffs)
        #spp = spectralanalysis.spectrum(p_coeffs)
        #sap = spectralanalysis.cross_spectrum(a_coeffs, p_coeffs)

        #admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)                                    
        admit, corr = spectralanalysis.SHBiasAdmitCorr(sap, saa, spp, tapers)
        end_spectrum = time.time()
        dur_spectrum_s = dur_spectrum_s + (end_spectrum - start_spectrum)


        for l in range(0, 10):                
            prob = spectralanalysis.SHConfidence(l, corr[l])                
            score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
            z_scores_intensity[i] = z_scores_intensity[i] + score  

        '''                  

        admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_range, saa_range, spp_range, tapers)                        
        for l in range(0, 10):                
            prob = spectralanalysis.SHConfidence(l, corr[l])                
            score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
            z_scores_range[i] = z_scores_range[i] + score

        admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_intensity, saa_intensity, spp_intensity, tapers)                            
        for l in range(0, 10):                
            prob = spectralanalysis.SHConfidence(l, corr[l])                
            score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
            z_scores_intensity[i] = z_scores_intensity[i] + score                           

        admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)                            
        for l in range(0, 10):                
            prob = spectralanalysis.SHConfidence(l, corr[l])                
            score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
            z_scores_image[i] = z_scores_image[i] + score                                                                   
        '''                  

    if (contains_match is not True):            
        continue


    #print(z_scores_range)
    #print(z_scores_intensity)
    #print(f'z_score > 2 = {np.sum(np.array(z_scores_range) > 3.8)} range, {np.sum(np.array(z_scores_intensity) > 20)} intensity')
    #print(f'true matches: {n_true_matches}')

    n_matches = n_matches + 1
    max_index_range, max_z_score_range = max(enumerate(z_scores_range), key=operator.itemgetter(1))
    max_index_intensity, max_z_score_intensity = max(enumerate(z_scores_intensity), key=operator.itemgetter(1))        
    max_index_image, max_z_score_image = max(enumerate(z_scores_image), key=operator.itemgetter(1))

    #print(f'max range: {max_z_score_range}, max intensity: {max_z_score_intensity}')
    max_index = max_index_range if max_z_score_range > max_z_score_intensity else max_index_intensity
    #max_index = max_index_intensity
    max_score = max_z_score_range if max_z_score_range > max_z_score_intensity else max_z_score_intensity

    max_index = max_index if max_score > max_z_score_image else max_index_image

    matching_index = nn_indices[max_index]
    dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
    if (dist <= max_pos_dist):
        loc_count = loc_count + 1;            
        #print('successful')
    #else:
        #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_score}.')            
        #matching_index = nn_indices[true_match_idx]
        #dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
        #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')

loc_recall = (loc_count*1.0) / n_matches    
loc_precision = (loc_count*1.0) / n_matches    
#print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
print(f'{loc_precision}')
#writer.add_scalar('Ext_Test/Precision/WindowedVoting', loc_precision, n_nearest_neighbors)
#print(f'Duration: {dur_neighbor_processing_s/n_test_set}s')    
print(f'Duration S^2 Transform: {dur_s2_s/n_test_set}s')
print(f'Duration Spectrum: {dur_spectrum_s/n_test_set}s')
